In [3]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 91.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 100.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 26.8 MB/s eta 0:00:00


In [4]:
from transformers import pipeline
generator = pipeline('text-generation',
                     model='huggingtweets/realdonaldtrump')
generator("My dream is", num_return_sequences=5)


/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1219: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'My dream is an economy where Jobs are going up 5%.'},
 {'generated_text': 'My dream is to win the election of Pennsylvania, and perhaps, maybe more importantly, to be President of the United States. We won 47% of the popular vote, and still have 47 Republicans to go. I have fought hard against the Republican establishment, and I still stand with them today!'},
 {'generated_text': 'My dream is that the Democrat Senators would eliminate the Obama HealthCare scam!'},
 {'generated_text': 'My dream is that the Supreme Court will take up the matter, and give us the Constitution. Now, we have seen enough of the political dead in the swing states - the 2020 Election.'},
 {'generated_text': 'My dream is a world where our countries and nations recognize our dignity as American citizens and let others honor our history. Biden was right to say last night that he wants to abolish the $2000 minimum wage, and he doesn’t want our cities to be run by fools anymore!'}]

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader

import numpy as np

import csv
import re
from collections import Counter


In [6]:
tweets = list(set(list(line[4] for line in csv.reader(open('/content/trump_insult_tweets_2014_to_2021.csv')))))
len(tweets)

5674

In [7]:
tweets[:10]

['Radical Left ANTIFA SCUM was easily rebuffed today by the big D.C. MAGA Rally crowd, only to return at night, after 99% of the crowd had left, to assault elderly people and families. Police got there, but late. Mayor is not doing her job!',
 'My son Donald did a good job last night. He was open, transparent and innocent. This is the greatest Witch Hunt in political history. Sad!',
 'Leakin’ Lyin’ James Comey! https://t.co/0qr5BcbRcl',
 'Mainstream media never covered Hillary’s massive ‘hacking’\n or coughing attack, yet it is #1 trending. What’s up?',
 'Another Fake Book by two third rate Washington Post reporters, has already proven to be inaccurately reported, to their great embarrassment, all for the purpose of demeaning and belittling a President who is getting great things done for our Country, at a record clip. Thank you!',
 'Hillary said "I really deplore the tone and inflammatory rhetoric of his campaign." I deplore the death and destruction she caused-stupidity',
 'Joe Biden

In [8]:
def preprocess(tweet_text):
    res = tweet_text.lower()
    res = re.sub(r'http\S+', '', res)
    res = re.sub(r'[^\w\s]', '', res)
    res = res.rstrip()
    res = res.lstrip()

    return res

In [9]:
tweets = [preprocess(t) for t in tweets]
tweets = [t for t in tweets if len(t) > 0]

In [10]:
len(tweets)

5673

In [11]:
tweets[:10]

['radical left antifa scum was easily rebuffed today by the big dc maga rally crowd only to return at night after 99 of the crowd had left to assault elderly people and families police got there but late mayor is not doing her job',
 'my son donald did a good job last night he was open transparent and innocent this is the greatest witch hunt in political history sad',
 'leakin lyin james comey',
 'mainstream media never covered hillarys massive hacking\n or coughing attack yet it is 1 trending whats up',
 'another fake book by two third rate washington post reporters has already proven to be inaccurately reported to their great embarrassment all for the purpose of demeaning and belittling a president who is getting great things done for our country at a record clip thank you',
 'hillary said i really deplore the tone and inflammatory rhetoric of his campaign i deplore the death and destruction she causedstupidity',
 'joe biden is the living embodiment of the corrupt political class tha

In [12]:
tweets_text = '<BOS> ' + ' <EOS> <BOS> '.join(tweets) + ' <EOS>'

In [13]:
tweets_text

'<BOS> radical left antifa scum was easily rebuffed today by the big dc maga rally crowd only to return at night after 99 of the crowd had left to assault elderly people and families police got there but late mayor is not doing her job <EOS> <BOS> my son donald did a good job last night he was open transparent and innocent this is the greatest witch hunt in political history sad <EOS> <BOS> leakin lyin james comey <EOS> <BOS> mainstream media never covered hillarys massive hacking\n or coughing attack yet it is 1 trending whats up <EOS> <BOS> another fake book by two third rate washington post reporters has already proven to be inaccurately reported to their great embarrassment all for the purpose of demeaning and belittling a president who is getting great things done for our country at a record clip thank you <EOS> <BOS> hillary said i really deplore the tone and inflammatory rhetoric of his campaign i deplore the death and destruction she causedstupidity <EOS> <BOS> joe biden is the

In [14]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, text, n):
        self.n = n
        self.text = text
        self.words = self.load_words()
        self.unique_tokens = self.get_unique_tokens()
        self.index_to_word = {index: word for index, word in enumerate(self.unique_tokens)}
        self.word_to_index = {word: index for index, word in enumerate(self.unique_tokens)}

        self.words_indexes = [self.word_to_index[w] for w in self.words]

    def load_words(self):
        return self.text.split(' ')

    def get_unique_tokens(self):
        word_counts = Counter(self.words)
        return sorted(word_counts, key=word_counts.get, reverse=True)

    def __len__(self):
        return len(self.words_indexes) - self.n

    def __getitem__(self, index):
        return (
            torch.tensor(self.words_indexes[index:index+self.n]),
            torch.tensor(self.words_indexes[index + 1 : index + self.n + 1]),
        )

In [15]:
d = Dataset(tweets_text, 4)

In [16]:
d.__getitem__(1)

(tensor([ 154,  120, 1626, 1989]), tensor([ 120, 1626, 1989,   22]))

In [17]:
class LSTM(nn.Module):
    def __init__(self, dataset):
        super(LSTM, self).__init__()
        self.lstm_size = 128
        self.embedding_dim = 128
        self.num_layers = 3

        n_vocab = len(dataset.unique_tokens)
        self.embedding = nn.Embedding(
            num_embeddings=n_vocab,
            embedding_dim=self.embedding_dim,
        )
        self.lstm = nn.LSTM(
            input_size=self.lstm_size,
            hidden_size=self.lstm_size,
            num_layers=self.num_layers,
            dropout=0.2,
        )
        self.fc = nn.Linear(self.lstm_size, n_vocab)

    def forward(self, x, prev_state):
        embedding_layer = self.embedding(x)
        output, state = self.lstm(embedding_layer, prev_state)
        logits = self.fc(output)
        return logits, state

    def init_state(self, sequence_length):
        return (torch.zeros(self.num_layers, sequence_length, self.lstm_size),
                torch.zeros(self.num_layers, sequence_length, self.lstm_size))

In [18]:
def train(dataset, model, n, n_epochs):
    model.train()

    dataloader = DataLoader(dataset, batch_size=256)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(n_epochs):
        hidden, current = model.init_state(n)

        for batch, (x, y) in enumerate(dataloader):
            optimizer.zero_grad()

            y_pred, (hidden, current) = model(x, (hidden, current))
            loss = criterion(y_pred.transpose(1, 2), y)

            hidden = hidden.detach()
            current = current.detach()

            loss.backward()
            optimizer.step()

            print({'epoch': epoch, 'batch': batch, 'loss': loss.item()})

        torch.save(model.state_dict(), 'checkpoint.pth')

In [19]:
def predict(dataset, model, start_text, next_words=50):
    model.eval()
    res_text = start_text.split(' ')
    hidden, current = model.init_state(len(res_text))

    for i in range(0, next_words):
        inp = torch.tensor([[dataset.word_to_index[w] for w in res_text[i:]]])
        y_pred, (hidden, current) = model(inp, (hidden, current))
        last_word_logits = y_pred[0][-1]

        prob = torch.nn.functional.softmax(last_word_logits, dim=0).detach().numpy()
        word_index = np.random.choice(len(last_word_logits), p=prob)

        word = dataset.index_to_word[word_index]

        if word == '<BOS>' or word == "<EOS>":
            break
        else:
            res_text.append(word)

    return ' '.join(res_text).lstrip('<BOS> ')

In [ ]:
dataset = Dataset(tweets_text, 4)
model = LSTM(dataset)

train(dataset, model, 4, n_epochs=10)


{'epoch': 0, 'batch': 0, 'loss': 9.211723327636719}
{'epoch': 0, 'batch': 1, 'loss': 9.211624145507812}
{'epoch': 0, 'batch': 2, 'loss': 9.194793701171875}
{'epoch': 0, 'batch': 3, 'loss': 9.19067668914795}
{'epoch': 0, 'batch': 4, 'loss': 9.18299388885498}
{'epoch': 0, 'batch': 5, 'loss': 9.17536735534668}
{'epoch': 0, 'batch': 6, 'loss': 9.15665340423584}
{'epoch': 0, 'batch': 7, 'loss': 9.13847827911377}
{'epoch': 0, 'batch': 8, 'loss': 9.110642433166504}
{'epoch': 0, 'batch': 9, 'loss': 9.065561294555664}
{'epoch': 0, 'batch': 10, 'loss': 8.984519004821777}
{'epoch': 0, 'batch': 11, 'loss': 8.8610258102417}
{'epoch': 0, 'batch': 12, 'loss': 8.692228317260742}
{'epoch': 0, 'batch': 13, 'loss': 8.455131530761719}
{'epoch': 0, 'batch': 14, 'loss': 8.317855834960938}
{'epoch': 0, 'batch': 15, 'loss': 8.186466217041016}
{'epoch': 0, 'batch': 16, 'loss': 7.8894195556640625}
{'epoch': 0, 'batch': 17, 'loss': 7.956984996795654}
{'epoch': 0, 'batch': 18, 'loss': 7.568058490753174}
{'epoch':

In [ ]:
print(predict(dataset, model, start_text='<BOS>'))